In [11]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os

In [12]:
# NOTE: Make sure to change the server name to whatever is applicable in your environment. If running locally, then use only ak.connect().
ak.connect("n119", 5555)

connected to arkouda server tcp://*:5555


In [13]:
src_list = [1, 2, 1, 3, 4, 2, 5, 4, 5, 6, 3, 8, 9, 4, 9, 10]
dst_list = [0, 1, 3, 4, 1, 4, 2, 5, 6, 7, 8, 9, 3, 9, 10, 4]

src_graph = ak.array(src_list)
dst_graph = ak.array(dst_list)


# Fixed attributes for subgraph
graph_node_ints = ak.array([10, 10, 10, 10, 10, 10, 10, 10,10, 10, 10])
graph_node_bools = ak.array([True, True, True, True, True, True, True, True, True, True, True])
graph_edge_ints = ak.array([5]* len(src_list))
graph_edge_bools = ak.array([True]*len(src_list))


# Subgraph dataframes
edge_df_h = ak.DataFrame({
    "src": src_graph,
    "dst": dst_graph,
    "rels1": graph_edge_ints,
    "rels2": graph_edge_bools
})

node_df_h = ak.DataFrame({
    "nodes": ak.array(list(set(src_list + dst_list))),
    "lbls2": graph_node_ints,
    "lbls3": graph_node_bools
})

# Create the subgraph
pg = ar.PropGraph()
pg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
pg.load_node_attributes(node_df_h, node_column="nodes")


NameError: name 'subgraph_edge_ints' is not defined

In [7]:


# Subgraph structure
src_list = [2, 3, 1, 3]
dst_list = [3, 1, 2, 0]
src_subgraph = ak.array(src_list)
dst_subgraph = ak.array(dst_list)


# Fixed attributes for subgraph
subgraph_node_ints = ak.array([10, 10, 10, 10])
subgraph_node_bools = ak.array([True, True, True, True])
subgraph_edge_ints = ak.array([5, 5, 5, 5])
subgraph_edge_bools = ak.array([True, True, True, True])


# Subgraph dataframes
edge_df_h = ak.DataFrame({
    "src": src_subgraph,
    "dst": dst_subgraph,
    "rels1": subgraph_edge_ints,
    "rels2": subgraph_edge_bools
})

node_df_h = ak.DataFrame({
    "nodes": ak.array(list(set(src_list + dst_list))),
    "lbls2": subgraph_node_ints,
    "lbls3": subgraph_node_bools
})

# Create the subgraph
sg = ar.PropGraph()
sg.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
sg.load_node_attributes(node_df_h, node_column="nodes")


In [8]:
"""VF2-SI STRUCTURAL + labels + rels"""

isos_as_vertices = ar.subgraph_isomorphism(pg, sg, 
                                           semantic_check = "and", algorithm_type = "si", 
                                           reorder_type = "structural", return_isos_as = "vertices")
print(f"We found {len(isos_as_vertices[0])/len(sg)} monos inside of the graph with reordering")

We found 1.0 monos inside of the graph with reordering


In [10]:
import networkx as nx
from networkx.algorithms import isomorphism

# Define the main graph
src_main = [1, 2, 1, 3, 4, 2, 5, 4, 5, 6, 3, 8, 9, 4, 9, 10]
dst_main = [0, 1, 3, 4, 1, 4, 2, 5, 6, 7, 8, 9, 3, 9, 10, 4]

labels2_maingraph = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]  # Integer labels for nodes
labels3_maingraph = [True, True, True, True, True, True, True, True, True, True, True]  # Boolean labels for nodes

main_graph = nx.DiGraph()
main_graph.add_edges_from(zip(src_main, dst_main))

# Add node attributes (integer and boolean labels)
for i, (int_label, bool_label) in enumerate(zip(labels2_maingraph, labels3_maingraph)):
    main_graph.nodes[i]["label_int"] = int_label
    main_graph.nodes[i]["label_bool"] = bool_label

# Add edge attributes (integer and boolean labels)
for edge in main_graph.edges:
    main_graph.edges[edge]["label_int"] = 5  # Assign integer label
    main_graph.edges[edge]["label_bool"] = True  # Assign boolean label

# Define the subgraph
src_list = [0, 1, 2, 1]
dst_list = [1, 2, 0, 3]

subgraph_node_ints = [10, 10, 10, 10]  # Integer labels for subgraph nodes
subgraph_node_bools = [True, True, True, True]  # Boolean labels for subgraph nodes

subgraph = nx.DiGraph()
subgraph.add_edges_from(zip(src_list, dst_list))

# Add node attributes (integer and boolean labels)
for i, (int_label, bool_label) in enumerate(zip(subgraph_node_ints, subgraph_node_bools)):
    subgraph.nodes[i]["label_int"] = int_label
    subgraph.nodes[i]["label_bool"] = bool_label

# Add edge attributes (integer and boolean labels)
for edge in subgraph.edges:
    subgraph.edges[edge]["label_int"] = 5  # Assign integer label
    subgraph.edges[edge]["label_bool"] = True  # Assign boolean label

# Define isomorphism conditions for nodes and edges
node_match = lambda n1, n2: n1["label_int"] == n2["label_int"] and n1["label_bool"] == n2["label_bool"]
edge_match = lambda e1, e2: e1["label_int"] == e2["label_int"] and e1["label_bool"] == e2["label_bool"]

# Find isomorphisms
gm = isomorphism.DiGraphMatcher(main_graph, subgraph, node_match=node_match, edge_match=edge_match)
isomorphism_count = 0

print("Found isomorphic subgraphs:")
for mapping in gm.subgraph_isomorphisms_iter():
    isomorphism_count += 1
    print(mapping)

print(f"Total number of isomorphisms: {isomorphism_count}")


Found isomorphic subgraphs:
{1: 0, 3: 1, 4: 2, 8: 3}
{2: 0, 4: 1, 5: 2, 9: 3}
{3: 0, 4: 1, 1: 2, 5: 3}
{3: 0, 4: 1, 9: 2, 5: 3}
{4: 0, 1: 1, 3: 2, 0: 3}
{4: 0, 5: 1, 2: 2, 6: 3}
{8: 0, 9: 1, 3: 2, 10: 3}
{10: 0, 4: 1, 9: 2, 1: 3}
{10: 0, 4: 1, 9: 2, 5: 3}
Total number of isomorphisms: 9
